In [23]:
import torch.nn as nn
import torch
torch.manual_seed(1773) 
import math

### Pytorch's way

In [24]:
embed_dim = 2
num_heads = 1
query = torch.tensor([[[0.7204, 0.0731],
         [0.9699, 0.1078],
         [0.8829, 0.4132]]])
key = query
value = query

multihead_attn = nn.MultiheadAttention(embed_dim, num_heads,batch_first=True,bias=True)
attn_output, attn_output_weights = multihead_attn(query, key, value)

### Getting the parameters

### Replicating Pytorch

In [36]:
embed_dim = 2
num_heads = 1
query = torch.tensor([[[0.7204, 0.0731],
         [0.9699, 0.1078],
         [0.8829, 0.4132]]])
key = query
value = query
weight = multihead_attn.state_dict()['in_proj_weight']

q,k, v =  query.matmul(weight.t()).chunk(3, dim=-1)

v = v.reshape(3,2)
q = q.reshape(3,2)
k = k.reshape(3,2)
q = q / math.sqrt(embed_dim)
attn1 = torch.matmul(q,k.T)
m = nn.Softmax(dim=-1)
my_attn_output_weights = m(attn1)
output = torch.matmul(my_attn_output_weights, v)
output = output.reshape((3,2))
out_proj = multihead_attn.state_dict()['out_proj.weight']
my_output = output.matmul(out_proj.t())